<a href="https://colab.research.google.com/github/MittalNeha/detr-panoptic/blob/main/dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In particular the COCO format [6] for panoptic segmentation consists in a PNG that stores the class-agnostic image segmentation and a JSON struct that stores the semantic information for each image segment. The unique id of the segment is used to retrieve the corresponding mask from the PNG while the category_id gives the semantic category. The isthing attribute distinguishes stuff and thing categories.
While the COCO dataset express segmentation as a list of vertex (that will be converted to the mask image) the Cityscapes script already outputs the PNG.

In [1]:
import os

In [ ]:
# !git clone https://github.com/facebookresearch/detr.git

Cloning into 'detr'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 247 (delta 0), reused 1 (delta 0), pack-reused 243
Receiving objects: 100% (247/247), 12.84 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [2]:
!git clone https://github.com/MittalNeha/Extensive_Vision_AI6.git
!cp -r /content/Extensive_Vision_AI6/CAPSTONE/detr/ .

Cloning into 'Extensive_Vision_AI6'...
remote: Enumerating objects: 549, done.
remote: Counting objects: 100% (549/549), done.
remote: Compressing objects: 100% (466/466), done.
remote: Total 549 (delta 251), reused 167 (delta 59), pack-reused 0
Receiving objects: 100% (549/549), 10.70 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# !wget https://drive.google.com/file/d/1IsK268zLnXB2Qq0X2LgNDwBZRuVwvjRx/view?usp=sharing

In [ ]:
!unzip -q /gdrive/MyDrive/EVA6/CAPSTONE/construction_materials_dataset.zip -d dataset

In [ ]:
%cd Extensive_Vision_AI6/CAPSTONE/

/content/Extensive_Vision_AI6/CAPSTONE


In [ ]:
!python CvattoCoco.py

In [ ]:
!zip -r /gdrive/MyDrive/EVA6/CAPSTONE/parsed_data.zip /content/content/parsed_data

In [ ]:
%cd ../../detr

/content/detr


#Create Panoptic mask.
1. pass custom dataset images through pretrained detr
2. Combine coco predictions with custom dataset ground truth

In [4]:
! pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-k7fgm7_v
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-k7fgm7_v
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=96bf09f7f589e14db2b15c8604fe2189ea7fc93c0f1ae06681021defd0f2c5c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-5glxzrjn/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [5]:
!unzip -q /gdrive/MyDrive/EVA6/CAPSTONE/parsed_data.zip

In [6]:
%cd detr

/content/detr


In [ ]:
!python create_panoptic_dataset.py --batch_size 2 --dataset_file coco_materials --data_path /content/content/content/parsed_data/ --device cuda --output_dir output/ --coco_panoptic_path /content/coco_panoptic

In [10]:
%cd ..

/content


In [ ]:
!zip -r /gdrive/MyDrive/EVA6/CAPSTONE/panoptic.zip coco_panoptic

In [10]:
!mv coco_panoptic/annotations/panoptic_val/ coco_panoptic/annotations/panoptic_materials_val/
!mv coco_panoptic/annotations/panoptic_train/ coco_panoptic/annotations/panoptic_materials_train/

In [6]:
!unzip -q /gdrive/MyDrive/EVA6/CAPSTONE/panoptic.zip 

In [10]:
!cp -r parsed_data/materials_test/ coco_panoptic/
!cp -r parsed_data/materials_val/ coco_panoptic/
!cp -r parsed_data/materials_train/ coco_panoptic/

###Also add some annotations from the coco val dataset 

In [2]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip

In [5]:
!wget http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
!unzip -q panoptic_annotations_trainval2017.zip
%cd annotations/
!unzip -q panoptic_train2017.zip
!unzip -q panoptic_val2017.zip
%cd ..

--2021-09-23 16:44:19--  http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.94.148
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.94.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 860725834 (821M) [application/zip]
Saving to: ‘panoptic_annotations_trainval2017.zip’

panoptic_annotation 100%[===================>] 820.85M  39.6MB/s    in 21s     

2021-09-23 16:44:40 (39.5 MB/s) - ‘panoptic_annotations_trainval2017.zip’ saved [860725834/860725834]

/content/annotations
/content


In [12]:
!python combine_panoptic.py

In [15]:
!cp combined_train.json coco_panoptic/annotations/train_panoptic.json
!cp combined_val.json coco_panoptic/annotations/val_panoptic.json

## Convert the json annotation file in coco_detection format for training the box model

In [18]:
!python converttococo.py

## Fix: "iscrowd" key is missing in panoptic json
TODO: fix in the code

In [21]:
import json
from copy import deepcopy

def hotfix_iscrowd(in_path):
    f = open(in_path)
    json_data = json.load(f)
    f.close()

    for ann in json_data['annotations']:
        for seg in ann['segments_info']:
            seg['iscrowd'] = 0

    out_file = open(in_path, "w")
    json.dump(json_data, out_file, indent = 4)
    out_file.close()

hotfix_iscrowd('coco_panoptic/annotations/train_panoptic.json')
hotfix_iscrowd('coco_panoptic/annotations/val_panoptic.json')

In [ ]:
!zip -r /gdrive/MyDrive/EVA6/CAPSTONE/panoptic.zip coco_panoptic

In [ ]:
!python create_panoptic_dataset.py --batch_size 1 --dataset_file coco_materials --data_path /content/content/content/parsed_data/ --device cuda --output_dir output/ --coco_panoptic_path /content/coco_panoptic

In [ ]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [ ]:
torch.cuda.device_count()

1

In [ ]:
!nvidia-smi

Wed Sep 22 13:56:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
!nvidia-smi

Thu Sep 23 17:30:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------